In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import regularizers

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedKFold, train_test_split



In [2]:
train_data=pd.read_csv("Train_Data.csv")
train_data

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,...,0.06,0.06,0.00,0.00,0.00,0.00,1.00,1.00,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,...,0.45,0.04,0.00,0.00,0.11,0.00,0.02,0.00,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,...,1.00,0.00,0.02,0.03,0.00,0.00,0.00,0.00,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,...,0.01,0.08,0.00,0.00,0.00,0.00,1.00,1.00,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86840,0,tcp,http,SF,216,674,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.01,0.00,0.00,21,0
86841,0,tcp,http,SF,368,1725,0,0,0,0,...,1.00,0.00,0.03,0.05,0.00,0.00,0.13,0.06,20,0
86842,28,tcp,ftp,SF,1483,4152,0,0,0,30,...,0.28,0.04,0.01,0.00,0.00,0.00,0.00,0.00,20,0
86843,0,tcp,smtp,SF,1679,385,0,0,0,0,...,0.16,0.08,0.00,0.00,0.69,0.00,0.01,0.00,18,0


In [3]:
encoder=OneHotEncoder()
encoded_data=encoder.fit_transform(train_data[['protocoltype','flag','service']])
dense_array = encoded_data.toarray()
encoded_df = pd.DataFrame(dense_array, columns=encoder.get_feature_names_out(['protocoltype','flag','service']))
encoded_df

,protocoltype_icmp,protocoltype_tcp,protocoltype_udp,flag_OTH,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_S1,flag_S2,...,service_telnet,service_tftp_u,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86840,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86841,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86842,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86843,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_data.drop(['protocoltype','flag','service'],axis=1,inplace=True)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(train_data)
scaled_df = pd.DataFrame(scaled_features, columns=train_data.columns)
scaled_df

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,-0.100328,-0.023605,-0.049289,-0.006787,0.0,-0.004553,-0.078877,-0.020846,-0.888462,-0.011452,...,-1.042709,0.094988,-0.363793,-0.293940,-0.699276,-0.688094,3.209750,3.229542,0.660945,1.277430
1,-0.100328,-0.020146,-0.042073,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,-0.170910,-0.098581,-0.363793,-0.293940,-0.462354,-0.688094,-0.256010,-0.323484,-1.449625,-0.782822
2,-0.100328,-0.022986,-0.032241,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,1.058551,-0.485721,-0.267818,0.248846,-0.699276,-0.688094,-0.326740,-0.323484,0.660945,-0.782822
3,-0.100328,-0.022949,-0.024393,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,1.058551,-0.485721,-0.363793,-0.293940,-0.699276,-0.688094,-0.326740,-0.323484,0.660945,-0.782822
4,-0.100328,-0.023605,-0.049289,-0.006787,0.0,-0.004553,-0.078877,-0.020846,-0.888462,-0.011452,...,-1.154478,0.288558,-0.363793,-0.293940,-0.699276,-0.688094,3.209750,3.229542,-0.746102,1.277430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86840,-0.100328,-0.023002,-0.037130,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,1.058551,-0.485721,-0.315805,0.248846,-0.677738,-0.666519,-0.326740,-0.323484,0.660945,-0.782822
86841,-0.100328,-0.022578,-0.018169,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,1.058551,-0.485721,-0.219830,0.610704,-0.699276,-0.688094,0.133004,-0.110303,-0.042578,-0.782822
86842,-0.073360,-0.019465,0.025616,-0.006787,0.0,-0.004553,16.243680,-0.020846,1.125541,-0.011452,...,-0.550925,-0.098581,-0.315805,-0.293940,-0.699276,-0.688094,-0.326740,-0.323484,-0.042578,-0.782822
86843,-0.100328,-0.018918,-0.042343,-0.006787,0.0,-0.004553,-0.078877,-0.020846,1.125541,-0.011452,...,-0.819170,0.288558,-0.363793,-0.293940,0.786869,-0.688094,-0.291375,-0.323484,-1.449625,-0.782822


In [5]:

train_data=pd.concat([scaled_df,encoded_df],axis=1)
train_data.shape

(86845, 118)

In [6]:
x=train_data.drop(['attack'],axis=1)
y=train_data['attack']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
model=Sequential()

model.add(Dense(500, activation='relu', input_dim=117))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.F1Score,
                       keras.metrics.Accuracy])


TypeError: 'property' object is not iterable

In [14]:
model.fit(x_train,y_train ,epochs=10)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [13]:
y_pred=model.predict(x_test)


815/815 [==============================] - 2s 2ms/step


In [15]:
Y_test=[]
for x in y_test:
    if x[0]==0:
        Y_test.append(1)
    else:
        Y_test.append(0)

Y_pred=[]
# print(y_pred)
for x in y_pred:
    if x[0]>x[1]:
        Y_pred.append(0)
    else:
        Y_pred.append(1)
        
print(Y_pred)
print(Y_test)


[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 

In [16]:
print(f1_score(Y_test,Y_pred))

0.0010857763300760044


In [17]:
test_data=pd.read_csv("Test_Data.csv")


In [18]:
print(set(train_data['service'].values)-set(test_data['service'].values))

KeyError: 'service'